**Bayesian Statistics Midterm Reference Guide**

---

## **1. Metropolis-Hastings Algorithm (20%)**
**Relevant Notebook:** Bayes_03_Metropolis_Hastings.ipynb (Q3)

### **Key Concepts:**
- **Goal:** Sample from a posterior distribution using a proposal distribution.
- **Steps:**
  1. Choose a **proposal distribution** $q(x' | x)$.
  2. Compute **acceptance probability**:
     $[ A = \min \left( 1, \frac{p(x') q(x | x')}{p(x) q(x' | x)} \right) ]$
  3. Accept \( x' \) with probability \( A \), else retain \( x \).

### **How Problems Were Solved in Homework:**
- Used a mixture of normal and uniform proposals for efficiency.
- Adjusted proposal variance to balance acceptance rate and exploration.
- Ensured proper initialization to avoid early rejections.

### **Code Snippet:**
```python
import numpy as np
import scipy.stats as stats

def metropolis_hastings(target_pdf, proposal_sampler, n_samples=1000, x_init=0):
    samples = [x_init]
    for _ in range(n_samples):
        x_new = proposal_sampler(samples[-1])
        accept_ratio = target_pdf(x_new) / target_pdf(samples[-1])
        if np.random.rand() < accept_ratio:
            samples.append(x_new)
        else:
            samples.append(samples[-1])
    return np.array(samples)
```

### **Potential Variations & How to Address Them:**
- **Different Proposal Distributions:**
  ```python
  def laplace_proposal(x):
      return np.random.laplace(x, scale=1.0)
  ```
- **Multivariate MH:**
  ```python
  def multivariate_proposal(x, cov_matrix):
      return np.random.multivariate_normal(x, cov_matrix)
  ```
- **Adaptive Metropolis:**
  ```python
  def adaptive_proposal(x, step_size):
      return np.random.normal(x, step_size)
  ```

### **Common Mistakes & Debugging Fixes:**
- **Low acceptance rate:** Tune step size dynamically:
  ```python
  step_size = 0.1
  if accept_ratio < 0.2:
      step_size *= 0.9
  elif accept_ratio > 0.8:
      step_size *= 1.1
  ```
- **Slow mixing:** Increase proposal range:
  ```python
  proposal_sampler = lambda x: np.random.normal(x, scale=2.0)
  ```
- **Local mode trapping:** Implement mixture proposals:
  ```python
  proposal_sampler = lambda x: np.random.choice([np.random.normal(x, 0.5), np.random.normal(x, 2.0)])
  ```

---

## **2. PyMC for Bayesian Inference (40%)**
**Relevant Notebooks:** Bayes_04_PyMC_Universal_Samplers.ipynb (Q2)

### **Key Concepts:**
- **PyMC Model Specification:** Define priors, likelihoods, and posterior sampling.
- **NUTS Sampler:** No-U-Turn Sampler (automatically selects step size).

### **How Problems Were Solved in Homework:**
- Defined hierarchical models and assessed sensitivity to priors.
- Diagnosed sampling efficiency using ArviZ diagnostics.
- Ensured proper chain mixing by running multiple chains.

### **Code Snippet (Fixing Common Issues):**
- **Increasing `target_accept` to prevent divergences:**
  ```python
  with model:
      trace = pm.sample(1000, target_accept=0.9, return_inferencedata=True)
  ```
- **Handling bad priors with prior predictive checks:**
  ```python
  with model:
      prior_checks = pm.sample_prior_predictive()
  az.plot_ppc(prior_checks)
  ```
- **Ensuring good convergence by increasing chains and tuning:**
  ```python
  with model:
      trace = pm.sample(2000, chains=4, tune=1000, return_inferencedata=True)
  ```

---

## **3. Gibbs Sampling (40%)**
**Relevant Notebook:** Bayes_02_The_Normal-Gamma_Model.ipynb (Q1, Q2)

### **Key Concepts:**
- **Iteratively sample each variable** conditioned on the others.
- **Metropolis-within-Gibbs:** Combines Metropolis-Hastings for non-conjugate updates.

### **How Problems Were Solved in Homework:**
- Used Normal-Gamma conjugacy for efficient sampling.
- Applied Metropolis steps for discrete parameters within Gibbs.
- Monitored chain convergence using trace plots.

### **Code Snippet (Fixing Common Issues):**
- **Fixing slow mixing with better initialization:**
  ```python
  mu = np.mean(x_data) + np.random.normal(0, 1)
  sigma2 = np.var(x_data) * np.random.gamma(2, 2)
  ```
- **Handling numerical instability using log-sampling:**
  ```python
  log_sigma2 = np.log(np.random.gamma(alpha_n, 1/beta_n))
  sigma2 = np.exp(log_sigma2)
  ```
- **Improving convergence using multiple chains:**
  ```python
  samples1 = gibbs_sampler(1000, x_data)
  samples2 = gibbs_sampler(1000, x_data)
  plt.plot(samples1[:, 0], label='Chain 1')
  plt.plot(samples2[:, 0], label='Chain 2')
  plt.legend()
  ```

---

## **4. General Debugging Tips:**
- **Autocorrelation Check:** High autocorrelation indicates poor mixing.
  ```python
  az.plot_autocorr(trace, var_names=["alpha", "beta"])
  ```
- **Burn-in Period:** Discard initial samples to ensure convergence.
  ```python
  trace_posterior = trace.sel(draw=slice(500, None))
  ```
- **Posterior Predictive Checks:** Use `az.plot_ppc(trace)` to validate model assumptions.
  ```python
  az.plot_ppc(az.from_pymc3(trace))
  ```
- **Alternative Sampling Methods:** If Gibbs is slow, try Hamiltonian Monte Carlo.
  ```python
  with model:
      trace = pm.sample(1000, step=pm.NUTS(), return_inferencedata=True)
  ```

---

This **expanded reference guide** now includes **code snippets for debugging solutions**, ensuring you have direct implementations for any potential midterm modifications!

